# README

Proviamo a usare solo le slice a percentile 100 per un singolo canale (es. T1).

Splittiamo il dataset in 2 (50% e 50%) e effettuiamo il training di una ResNet singolo canale.

Facciamo con 60 epoche per portare a convergenza.

Utilizziamo poi il modello trainato per generare features sui dati di test.

Le features di test le usiamo per effettuare il training di una SVM (split 80%/20%).

Valutiamo la bontà della SVM sul restante 20% delle features di test.

In [1]:
!pip install --user boruta 

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pickle
import numpy as np
import pandas as pd

class Dataset():
    
    def __init__(self, dataset_path):
        
        with open(dataset_path, "rb") as file:
            self.X, self.y = pickle.load(file)
            
        subjects = np.array(list(self.X.keys()))
        
        self.slices = np.concatenate([self.X[subject] for subject in subjects])
        self.labels = np.concatenate([np.repeat((self.y)[subject], (self.X)[subject].shape[0]) for subject in subjects])
        self.subjects = np.concatenate([np.repeat(subject, (self.X)[subject].shape[0]) for subject in subjects])

    def get_subjects(self):
        return list(self.X.keys())

In [3]:
dataset = Dataset("dataset-survivor-t1-adc-flair-224-100-perc.pickle")

In [4]:
subjects = np.array(dataset.get_subjects())

In [5]:
from sklearn.model_selection import train_test_split

random_state = 42

train_index, test_index = train_test_split(list(range(subjects.shape[0])), test_size=0.2, random_state=random_state)

# Train the Model

In [6]:
from tensorflow.keras.applications import *
from keras import backend as K

model_name = "ResNet50"

IMG_SHAPE = (224, 224, 3)

K.clear_session()

K.set_image_data_format('channels_last')

base_model = eval(model_name)(weights=None, include_top=False, input_shape=IMG_SHAPE)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Using TensorFlow backend.


In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

# add a global spatial average pooling layer
x = base_model.output

x = GlobalAveragePooling2D()(x)

#drop-out
x = Dropout(0.5)(x)

# Feature leayer
prediction_layer = Dense(512, activation='relu')(x)

x = Dropout(0.5)(prediction_layer)

# and a logistic layer -- let's say we have 200 classes
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [8]:
import tensorflow as tf

# rmsprop = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

# from keras.optimizers import SGD, Adam

# gd = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9, decay=0.001/10, nesterov=False)

adam = tf.keras.optimizers.Adam(lr=0.0001)

# model.compile(optimizer= gd, loss='categorical_crossentropy', metrics=['accuracy'])

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=adam, loss="binary_crossentropy", metrics=['accuracy', 'binary_crossentropy'])

In [9]:
import tensorflow as tf

categorical_labels = tf.keras.utils.to_categorical(dataset.labels)

In [10]:
X_train = dataset.slices[np.isin(dataset.subjects, np.array(subjects)[train_index])]

In [11]:
X_test = dataset.slices[np.isin(dataset.subjects, np.array(subjects)[test_index])]

In [12]:
y_train = categorical_labels[np.isin(dataset.subjects, np.array(subjects)[train_index])]

In [13]:
y_test = categorical_labels[np.isin(dataset.subjects, np.array(subjects)[test_index])]

In [14]:
subjects_test = dataset.subjects[np.isin(dataset.subjects, np.array(subjects)[test_index])]

In [15]:
labels_test = dataset.labels[np.isin(dataset.subjects, np.array(subjects)[test_index])]

In [16]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((103, 224, 224, 3), (27, 224, 224, 3), (103, 2), (27, 2))

In [17]:
epochs = 1000  # Increase this value for better results (i.e., more training)

batch_size = 16  # Increasing this value might speed up fitting

# Data Augmentation

In [18]:
import tensorflow as tf

datagen = tf.keras.preprocessing.image.ImageDataGenerator(    
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    # shear_range=0.2,
    # zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train)

In [ ]:
import numpy as np

# Fit
fit = model.fit(
        datagen.flow(X_train, y_train, batch_size=batch_size), # X_train, y_train, 
        epochs=epochs,
        # callbacks=[tensorboard_callback, early_stopping_callback],
        validation_data=(X_test, y_test), 
        shuffle=True) #,
        # batch_size=batch_size)

Epoch 1/1000
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
7/7 [==============================] - 40s 6s/step - loss: 1.3896 - acc: 0.5243 - binary_crossentropy: 1.3317 - val_loss: 0.6932 - val_acc: 0.5556 - val_binary_crossentropy: 0.6893
Epoch 2/1000
7/7 [==============================] - 1s 170ms/step - loss: 1.3158 - acc: 0.5825 - binary_crossentropy: 1.3391 - val_loss: 0.6936 - val_acc: 0.5556 - val_binary_crossentropy: 0.6902
Epoch 3/1000
7/7 [==============================] - 4s 565ms/step - loss: 1.1131 - acc: 0.5243 - binary_crossentropy: 1.1040 - val_loss: 0.6927 - val_acc: 0.5556 - val_binary_crossentropy: 0.6893
Epoch 4/1000
7/7 [==============================] - 4s 609ms/step - loss: 1.3251 - acc: 0.4078 - binary_crossentropy: 1.2894 - val_loss: 0.6932 - val_acc: 0.5556 - val_binary_crossentropy: 0.6883
Epoch 5/1000
7/7 [==============================] - 4s 596ms/step - loss: 1.3465 - acc: 0.4757 - binary_crossentropy: 1.2794

7/7 [==============================] - 4s 577ms/step - loss: 0.8183 - acc: 0.6117 - binary_crossentropy: 0.8393 - val_loss: 1.0055 - val_acc: 0.4444 - val_binary_crossentropy: 1.0770
Epoch 17/1000
7/7 [==============================] - 5s 657ms/step - loss: 1.1414 - acc: 0.5340 - binary_crossentropy: 1.0986 - val_loss: 1.1184 - val_acc: 0.4444 - val_binary_crossentropy: 1.1219
Epoch 18/1000
7/7 [==============================] - 5s 674ms/step - loss: 0.8252 - acc: 0.6408 - binary_crossentropy: 0.8017 - val_loss: 1.0061 - val_acc: 0.4444 - val_binary_crossentropy: 1.0776
Epoch 19/1000
7/7 [==============================] - 4s 508ms/step - loss: 0.7641 - acc: 0.6408 - binary_crossentropy: 0.7923 - val_loss: 0.9620 - val_acc: 0.4444 - val_binary_crossentropy: 1.0276
Epoch 20/1000
7/7 [==============================] - 4s 528ms/step - loss: 1.1883 - acc: 0.4660 - binary_crossentropy: 1.1682 - val_loss: 0.9727 - val_acc: 0.4444 - val_binary_crossentropy: 1.0397
Epoch 21/1000
7/7 [==========

7/7 [==============================] - 4s 591ms/step - loss: 0.8400 - acc: 0.6117 - binary_crossentropy: 0.8583 - val_loss: 1.3850 - val_acc: 0.4444 - val_binary_crossentropy: 1.4994
Epoch 33/1000
7/7 [==============================] - 4s 544ms/step - loss: 0.8308 - acc: 0.6505 - binary_crossentropy: 0.7653 - val_loss: 1.4727 - val_acc: 0.4444 - val_binary_crossentropy: 1.5963
Epoch 34/1000
7/7 [==============================] - 3s 409ms/step - loss: 0.8572 - acc: 0.6019 - binary_crossentropy: 0.8432 - val_loss: 1.3164 - val_acc: 0.4444 - val_binary_crossentropy: 1.4266
Epoch 35/1000
7/7 [==============================] - 5s 700ms/step - loss: 0.6296 - acc: 0.7184 - binary_crossentropy: 0.6211 - val_loss: 0.9012 - val_acc: 0.4444 - val_binary_crossentropy: 0.9601
Epoch 36/1000
7/7 [==============================] - 3s 490ms/step - loss: 0.8000 - acc: 0.6117 - binary_crossentropy: 0.8100 - val_loss: 1.0300 - val_acc: 0.4444 - val_binary_crossentropy: 1.1080
Epoch 37/1000
7/7 [==========

7/7 [==============================] - 4s 562ms/step - loss: 0.6689 - acc: 0.6311 - binary_crossentropy: 0.6514 - val_loss: 1.2267 - val_acc: 0.3333 - val_binary_crossentropy: 1.2328
Epoch 49/1000
7/7 [==============================] - 5s 669ms/step - loss: 0.6879 - acc: 0.6893 - binary_crossentropy: 0.6588 - val_loss: 1.6102 - val_acc: 0.4444 - val_binary_crossentropy: 1.7363
Epoch 50/1000
7/7 [==============================] - 4s 620ms/step - loss: 0.6748 - acc: 0.6311 - binary_crossentropy: 0.6919 - val_loss: 1.1566 - val_acc: 0.3704 - val_binary_crossentropy: 1.2398
Epoch 51/1000
7/7 [==============================] - 5s 743ms/step - loss: 0.5736 - acc: 0.6699 - binary_crossentropy: 0.5683 - val_loss: 1.2386 - val_acc: 0.3333 - val_binary_crossentropy: 1.3301
Epoch 52/1000
7/7 [==============================] - 6s 795ms/step - loss: 0.5011 - acc: 0.7573 - binary_crossentropy: 0.5008 - val_loss: 2.4243 - val_acc: 0.4444 - val_binary_crossentropy: 2.6285
Epoch 53/1000
7/7 [==========

7/7 [==============================] - 5s 713ms/step - loss: 0.5760 - acc: 0.7573 - binary_crossentropy: 0.5402 - val_loss: 1.1665 - val_acc: 0.5185 - val_binary_crossentropy: 1.2250
Epoch 65/1000
7/7 [==============================] - 5s 710ms/step - loss: 0.4927 - acc: 0.7379 - binary_crossentropy: 0.5073 - val_loss: 1.2057 - val_acc: 0.5556 - val_binary_crossentropy: 1.2616
Epoch 66/1000
7/7 [==============================] - 5s 707ms/step - loss: 0.5022 - acc: 0.6699 - binary_crossentropy: 0.5082 - val_loss: 1.2113 - val_acc: 0.4444 - val_binary_crossentropy: 1.2707
Epoch 67/1000
7/7 [==============================] - 4s 571ms/step - loss: 0.4405 - acc: 0.7864 - binary_crossentropy: 0.4460 - val_loss: 1.2923 - val_acc: 0.4815 - val_binary_crossentropy: 1.3315
Epoch 68/1000
7/7 [==============================] - 5s 664ms/step - loss: 0.5186 - acc: 0.7670 - binary_crossentropy: 0.5017 - val_loss: 1.2918 - val_acc: 0.4444 - val_binary_crossentropy: 1.3353
Epoch 69/1000
7/7 [==========

...

7/7 [==============================] - 5s 714ms/step - loss: 0.2625 - acc: 0.9029 - binary_crossentropy: 0.2674 - val_loss: 3.2142 - val_acc: 0.4815 - val_binary_crossentropy: 3.4737
Epoch 417/1000
7/7 [==============================] - 4s 581ms/step - loss: 0.2033 - acc: 0.9126 - binary_crossentropy: 0.2155 - val_loss: 2.7971 - val_acc: 0.4815 - val_binary_crossentropy: 3.0308
Epoch 418/1000
7/7 [==============================] - 4s 592ms/step - loss: 0.1658 - acc: 0.9126 - binary_crossentropy: 0.1700 - val_loss: 3.1546 - val_acc: 0.4815 - val_binary_crossentropy: 3.4288
Epoch 419/1000
7/7 [==============================] - 6s 787ms/step - loss: 0.2229 - acc: 0.9320 - binary_crossentropy: 0.1867 - val_loss: 3.6620 - val_acc: 0.4815 - val_binary_crossentropy: 3.9831
Epoch 420/1000
7/7 [==============================] - 5s 692ms/step - loss: 0.3326 - acc: 0.9223 - binary_crossentropy: 0.3237 - val_loss: 3.3544 - val_acc: 0.4815 - val_binary_crossentropy: 3.6419
Epoch 421/1000
7/7 [=====

7/7 [==============================] - 3s 487ms/step - loss: 0.1447 - acc: 0.9709 - binary_crossentropy: 0.1535 - val_loss: 1.8654 - val_acc: 0.5926 - val_binary_crossentropy: 1.8806
Epoch 433/1000
7/7 [==============================] - 4s 613ms/step - loss: 0.2616 - acc: 0.8641 - binary_crossentropy: 0.2589 - val_loss: 1.7477 - val_acc: 0.5185 - val_binary_crossentropy: 1.8052
Epoch 434/1000
7/7 [==============================] - 5s 678ms/step - loss: 0.4320 - acc: 0.8447 - binary_crossentropy: 0.4611 - val_loss: 1.7407 - val_acc: 0.5185 - val_binary_crossentropy: 1.8091
Epoch 435/1000
7/7 [==============================] - 5s 707ms/step - loss: 0.2457 - acc: 0.9029 - binary_crossentropy: 0.2317 - val_loss: 1.4189 - val_acc: 0.4815 - val_binary_crossentropy: 1.4197
Epoch 436/1000
7/7 [==============================] - 4s 557ms/step - loss: 0.2974 - acc: 0.8641 - binary_crossentropy: 0.2930 - val_loss: 1.5226 - val_acc: 0.5926 - val_binary_crossentropy: 1.5051
Epoch 437/1000
7/7 [=====

7/7 [==============================] - 4s 585ms/step - loss: 0.1312 - acc: 0.9612 - binary_crossentropy: 0.1372 - val_loss: 1.7165 - val_acc: 0.5926 - val_binary_crossentropy: 1.7483
Epoch 449/1000
7/7 [==============================] - 5s 720ms/step - loss: 0.1579 - acc: 0.9515 - binary_crossentropy: 0.1481 - val_loss: 1.8539 - val_acc: 0.5185 - val_binary_crossentropy: 1.9111
Epoch 450/1000
7/7 [==============================] - 4s 588ms/step - loss: 0.1620 - acc: 0.9417 - binary_crossentropy: 0.1676 - val_loss: 2.1055 - val_acc: 0.5556 - val_binary_crossentropy: 2.1549
Epoch 451/1000
7/7 [==============================] - 5s 681ms/step - loss: 0.1865 - acc: 0.9126 - binary_crossentropy: 0.1837 - val_loss: 2.3392 - val_acc: 0.5185 - val_binary_crossentropy: 2.4444
Epoch 452/1000
7/7 [==============================] - 4s 513ms/step - loss: 0.2709 - acc: 0.9320 - binary_crossentropy: 0.2283 - val_loss: 2.3536 - val_acc: 0.5185 - val_binary_crossentropy: 2.4736
Epoch 453/1000
7/7 [=====

7/7 [==============================] - 5s 665ms/step - loss: 0.1195 - acc: 0.9515 - binary_crossentropy: 0.1147 - val_loss: 2.1538 - val_acc: 0.5926 - val_binary_crossentropy: 2.1893
Epoch 465/1000
7/7 [==============================] - 4s 634ms/step - loss: 0.2130 - acc: 0.9320 - binary_crossentropy: 0.2275 - val_loss: 2.1114 - val_acc: 0.5926 - val_binary_crossentropy: 2.0846
Epoch 466/1000
7/7 [==============================] - 4s 595ms/step - loss: 0.1414 - acc: 0.9709 - binary_crossentropy: 0.1471 - val_loss: 2.1031 - val_acc: 0.5926 - val_binary_crossentropy: 2.0927
Epoch 467/1000
7/7 [==============================] - 5s 717ms/step - loss: 0.1420 - acc: 0.9612 - binary_crossentropy: 0.1497 - val_loss: 2.3871 - val_acc: 0.5185 - val_binary_crossentropy: 2.4049
Epoch 468/1000
7/7 [==============================] - 5s 671ms/step - loss: 0.1598 - acc: 0.9515 - binary_crossentropy: 0.1308 - val_loss: 2.5172 - val_acc: 0.4444 - val_binary_crossentropy: 2.6482
Epoch 469/1000
7/7 [=====

7/7 [==============================] - 5s 721ms/step - loss: 0.1114 - acc: 0.9709 - binary_crossentropy: 0.0943 - val_loss: 4.2995 - val_acc: 0.5185 - val_binary_crossentropy: 4.0848
Epoch 481/1000
7/7 [==============================] - 6s 858ms/step - loss: 0.2101 - acc: 0.9417 - binary_crossentropy: 0.1793 - val_loss: 3.6297 - val_acc: 0.5185 - val_binary_crossentropy: 3.4765
Epoch 482/1000
1/7 [===>..........................] - ETA: 2s - loss: 0.0210 - acc: 1.0000 - binary_crossentropy: 0.0210

In [23]:
fit

In [22]:
model.evaluate(X_train, y_train)

103/103 [==============================] - 1s 8ms/sample - loss: 4.5452 - acc: 0.4563 - binary_crossentropy: 4.5452


[4.5451609185598425, 0.4563107, 4.545161]

In [ ]:
# this is the model we will train
feature_model = Model(inputs=base_model.input, outputs=prediction_layer)

In [ ]:
y_features = feature_model.predict([X_test])

In [ ]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

"""
BORUTA Feature Selection
"""
def apply_boruta(X, Y):
    
    rf = RandomForestClassifier(n_jobs=-1, class_weight=None) 

    feature_selector = BorutaPy(rf)

    feature_selector.fit(X, Y)

    return X[:,feature_selector.ranking_ <= 70]


In [ ]:
sel_feature_sorted = apply_boruta(y_features, labels_test)

In [ ]:
print("Before Boruta:", y_features.shape)
print("After Boruta:", sel_feature_sorted.shape)

In [ ]:
from sklearn import svm

scores = []

for i in range(10):

    # Train and test split
    train_svm_index, test_svm_index = train_test_split(list(range(subjects_test.shape[0])), test_size=0.2, random_state=i)

    # Split dataset
    X_svm_train = sel_feature_sorted[np.isin(subjects_test, subjects_test[train_svm_index])]
    X_svm_test = sel_feature_sorted[np.isin(subjects_test, subjects_test[test_svm_index])]
    y_svm_train = labels_test[np.isin(subjects_test, subjects_test[train_svm_index])]
    y_svm_test = labels_test[np.isin(subjects_test, subjects_test[test_svm_index])]
    
    clf = svm.SVC()
    clf.fit(X_svm_train, y_svm_train)
    score = clf.score(X_svm_test, y_svm_test)
    print(score)
    scores.append(score)
    
print("--------------------------------")
print("Avg/Std:", np.average(scores), np.std(scores))

In [ ]:
# Results

T1:
- loss: 0.2955 
- acc: 0.9265
- binary_crossentropy: 0.2955
- val_loss: 1.4039
- val_acc: 0.5455
- val_binary_crossentropy: 1.4039
    
SVM score (avg on 10): 0.5714285714285714